---
title: CDAS
---

Package for accessing the Coordinate Data Analysis System (CDAS) https://cdaweb.gsfc.nasa.gov web services.

For example, see [notebook](../examples/01_cdas.ipynb).

In [ ]:
# | default_exp utils/cdas
# | export
from cdasws import CdasWs
from loguru import logger
from cdasws.datarepresentation import DataRepresentation
import xarray as xr
import polars as pl
from space_analysis.core import Dataset as V
from pydantic import ConfigDict, model_validator

In [ ]:
# | export
cdas = CdasWs()

In [ ]:
# | export
def get_dataset_variables(dataset: str):
    variables_dict = cdas.get_variables(dataset)
    for variable in variables_dict:
        logger.info(variable["Name"], variable["LongDescription"])

    return [variable["Name"] for variable in variables_dict]


def get_data(dataset, timerange, variables: list = None) -> xr.Dataset:
    variables = variables or get_dataset_variables(dataset)

    _, data = cdas.get_data(
        dataset,
        variables=variables,
        time0=timerange[0],
        time1=timerange[1],
        DataRepresentation=DataRepresentation.XARRAY,
    )

    return data

In [ ]:
# | export
class Variables(V):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    data: xr.Dataset = None

    # initize products from provider and dataset if not provided
    @model_validator(mode="after")
    def check_products(self):
        if self.parameters is None:
            self.parameters = get_dataset_variables(self.dataset)

    def retrieve_data(self):
        self.data = get_data(self.dataset, self.timerange, self.parameters)
        return self

    def get_data(self):
        if self.data is None:
            self.retrieve_data()
        return self.data

    def to_polars(self):
        return pl.DataFrame(self.to_pandas().reset_index())

    def to_pandas(self):
        return self.get_data().to_pandas()